# Find KEGG associations

This notebook will create a table that has the KEGG pathways that are associated with all genes, but we are particularly interested in those that are associated with the most and least stable genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_out_filename = "pao1_core_similarity_associations_spell.tsv"
pa14_out_filename = "pa14_core_similarity_associations_spell.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_core_similarity_expression_stats_spell.tsv"
pa14_similarity_scores_filename = "pa14_core_similarity_expression_stats_spell.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
print(pao1_similarity_scores.shape)
pao1_similarity_scores.head()

(5349, 14)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression
PAO1 id,,,,,,,,,,,,,,
PA2763,PA14_28370,0.223647,1.285228e-61,NaN,NaN,19.694848,24.743003,0.000000,5.391723,12.263734,23.681063,278.097141,612.216209,278.097141
PA0687,PA14_55430,0.186869,3.120286e-43,hxcS,NaN,50.518187,124.934061,0.000000,2.059509,10.258957,31.659012,1167.426739,15608.519591,1167.426739
PA0048,PA14_00600,0.172591,4.860103e-37,NaN,NaN,86.414619,96.944680,0.000000,25.523418,53.040761,113.420860,823.451891,9398.270927,823.451891
PA2363,PA14_34100,0.397806,2.600695e-202,hsiJ3,NaN,327.426738,370.231163,0.000000,69.472696,210.784639,457.708200,3176.807775,137071.114318,3176.807775
PA1171,PA14_49280,0.422021,4.773292e-230,sltB2,NaN,298.032395,158.892450,14.169995,193.150705,276.005155,374.746932,1019.941686,25246.810693,1005.771691


In [5]:
print(pa14_similarity_scores.shape)
pa14_similarity_scores.head()

(5347, 14)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression
PA14 id,,,,,,,,,,,,,,
PA14_54660,PA0744,0.416944,5.865823e-224,NaN,NaN,803.543344,1081.817170,14.240564,142.068583,414.380788,913.522137,6057.518156,1.170328e+06,6043.277593
PA14_30620,PA2588,0.494403,0.000000e+00,NaN,NaN,892.601508,1095.977874,15.326451,209.813434,484.465843,1002.523013,7104.684725,1.201168e+06,7089.358274
PA14_68400,PA5178,0.520295,0.000000e+00,NaN,NaN,3134.472215,3559.745121,25.280375,695.070834,2015.410297,4014.206074,25298.790957,1.267179e+07,25273.510582
PA14_38850,PA1983,0.239950,6.754510e-71,exaB,NaN,35.467321,219.499403,0.000000,0.000000,3.238127,11.433134,4543.642316,4.817999e+04,4543.642316
PA14_04760,PA0363,0.465735,3.828645e-286,coaD,NaN,355.338340,255.923231,19.537134,198.752170,284.643548,436.779949,2042.651981,6.549670e+04,2023.114847


In [6]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [7]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA2011, PA1736, PA4785, PA2003, PA1999, PA255..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA1737, PA1027, PA1736, PA4785, PA2550, PA087..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA1737, PA1027, PA4899, PA3426, PA1821, PA247..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA1737, PA1027, PA1736, PA4163, PA4785, PA257..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA4043, PA1736, PA4557, PA3652, PA4785, PA456..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [8]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [9]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for all genes

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
# Get KEGG associations for all genes in PAO1
all_pao1_gene_ids = list(pao1_similarity_scores.index)
pao1_associations = get_associated_pathways(all_pao1_gene_ids)

In [12]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 1)


,pathways present
gene id,
PA2763,[]
PA0687,[KEGG-Pathway-pae03070: Bacterial secretion sy...
PA0048,[]
PA2363,[KEGG-Module-M00334: Type VI secretion system]
PA1171,[]


In [13]:
# Map PA14 gene ids
pa14_associations = pao1_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_associations.set_index("PA14_ID", inplace=True)
print(pa14_associations.shape)
pa14_associations.head()

(5349, 1)


,pathways present
PA14_ID,
PA14_28370,[]
PA14_55430,[KEGG-Pathway-pae03070: Bacterial secretion sy...
PA14_00600,[]
PA14_34100,[KEGG-Module-M00334: Type VI secretion system]
PA14_49280,[]


In [14]:
# Merge KEGG associations with transcriptional similarity information
pao1_associations = pao1_similarity_scores.merge(
    pao1_associations, left_index=True, right_index=True, how="left"
)
pa14_associations = pa14_similarity_scores.merge(
    pa14_associations, left_index=True, right_index=True, how="left"
)

In [15]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 15)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present
PAO1 id,,,,,,,,,,,,,,,
PA2763,PA14_28370,0.223647,1.285228e-61,NaN,NaN,19.694848,24.743003,0.000000,5.391723,12.263734,23.681063,278.097141,612.216209,278.097141,[]
PA0687,PA14_55430,0.186869,3.120286e-43,hxcS,NaN,50.518187,124.934061,0.000000,2.059509,10.258957,31.659012,1167.426739,15608.519591,1167.426739,[KEGG-Pathway-pae03070: Bacterial secretion sy...
PA0048,PA14_00600,0.172591,4.860103e-37,NaN,NaN,86.414619,96.944680,0.000000,25.523418,53.040761,113.420860,823.451891,9398.270927,823.451891,[]
PA2363,PA14_34100,0.397806,2.600695e-202,hsiJ3,NaN,327.426738,370.231163,0.000000,69.472696,210.784639,457.708200,3176.807775,137071.114318,3176.807775,[KEGG-Module-M00334: Type VI secretion system]
PA1171,PA14_49280,0.422021,4.773292e-230,sltB2,NaN,298.032395,158.892450,14.169995,193.150705,276.005155,374.746932,1019.941686,25246.810693,1005.771691,[]


In [16]:
print(pa14_associations.shape)
pa14_associations.head()

(5347, 15)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present
PA14 id,,,,,,,,,,,,,,,
PA14_54660,PA0744,0.416944,5.865823e-224,NaN,NaN,803.543344,1081.817170,14.240564,142.068583,414.380788,913.522137,6057.518156,1.170328e+06,6043.277593,[]
PA14_30620,PA2588,0.494403,0.000000e+00,NaN,NaN,892.601508,1095.977874,15.326451,209.813434,484.465843,1002.523013,7104.684725,1.201168e+06,7089.358274,[]
PA14_68400,PA5178,0.520295,0.000000e+00,NaN,NaN,3134.472215,3559.745121,25.280375,695.070834,2015.410297,4014.206074,25298.790957,1.267179e+07,25273.510582,[]
PA14_38850,PA1983,0.239950,6.754510e-71,exaB,NaN,35.467321,219.499403,0.000000,0.000000,3.238127,11.433134,4543.642316,4.817999e+04,4543.642316,[]
PA14_04760,PA0363,0.465735,3.828645e-286,coaD,NaN,355.338340,255.923231,19.537134,198.752170,284.643548,436.779949,2042.651981,6.549670e+04,2023.114847,[KEGG-Pathway-pae00770: Pantothenate and CoA b...


In [17]:
# Save
pao1_associations.to_csv(pao1_out_filename, sep="\t")
pa14_associations.to_csv(pa14_out_filename, sep="\t")